## How the algorithm works
There are two possible outcomes of the algorithm:
1. **YES** -> We have enough guards.
2. **NO** -> We don't have enough guards.

### 1. How do we know that we have enough guards?
We know that we have enough guards, if **all** students were able to enter the university.

### 2. How do we know that we don't have enough guards?
We know that we don't have enough guards if we have to remove a guard from a gate that **still** has students waiting.

### Components
From the problem definition, we can identify three components of the algorithm:
1. Gates
2. Students
3. Guards

The diagram below shows the three components for the first example input
```
4 5 1
1 1 3 3 3
```
There are 4 gates (orange), 5 students (green), and 1 guard (blue). The students are queued up in front of their respective gates in ascending order, so that they enter in the correct order. The first two students `1 1`, are labelled `Student 1, Student 2`, and queue in front of gate 1. The next three students, `3 3 3` are labelled `Student 3, Student 4, Student 5`, and are queueing in front of gate 3. There is also a single guard in the guard pool.

![algorithm_overview](docs/algorithmic_question_overview.svg)

## Imports

In [2]:
from domain import Gate
from queue import PriorityQueue

## Read inputs
We will read from a file.

In [5]:
file_path = 'data/students_yes.txt'

with open(file_path, 'r') as file:
    n_gates, n_students, n_guards = map(int, file.readline().split())
    student_gates = list(map(int, file.readline().split()))

## 1. Gates
We can think of both the guards and students being assigned to gates. Students need to enter a specific gate and guards need to supervise an assigned gate. For this reason, we decided to create a `Gate` class that will hold students queueing for the gate as well as the guard that is supervising the gate. The class will help to facilitate interactions between all three components and suppor the logic of the overall algorithm. 

Gate class [documentation](https://jonasbarth.github.io/uni/msc/year/1/adm/hw/4/docs/gate.html).

Here, we create a mapping from gate numbers to gates.

In [7]:
gates = {gate_number: Gate(gate_number) for gate_number in range(1, n_gates + 1)}

In [15]:
def are_students_left(gates):
    """Helper method to check whether there are students waiting to be let in at any gate.
    
    :arg
    gates - a list of Gate objects.
    """
    return any(map(lambda gate: not gate.can_be_closed(), gates))   

## 2. Students
This is a list of integers $[1..M]$. Each student (integer) will be placed into a FIFO queue of a gate.

In [8]:
for student, student_gate in zip(range(n_students), student_gates):
    gates[student_gate].add_student(student)

## 3. Guards
We have a pool of $G$ guards that is represented as a list of integers $[1..G]$. We remove a guard from the list when we have a gate where students are waiting but no supervising guard. We add a guard when all students at the gate that the guard is supervising, have been let in.

In [14]:
guards = list(range(1, n_guards + 1))

## Algorithm
### The idea behind the algorithm is that
* we find the gate where the next student will be let in.
* if the gate has a guard, we can let the student in.
* if the gate does not have a guard, we try to assign one. If there are no free guards, we know that don't have enough guards because all guards are supervising gates where students are still waiting.

Here are the steps of the algorithm.
```
while students are waiting to be let in:
    get the gate where the next student will be let in.
    
    if the gate has a guard:
        let in the next student
        
    if the gate has no guard:
        if there are free guards in the pool:
            assign a guard from the pool.
            let in the next student.
            
        if there are no free guards:
            break the loop because all guards are occupied
            
    if the gate has no more students waiting:
        remove guard and put back into the guard pool.
```

In [10]:
queue = PriorityQueue(maxsize=n_gates)
for gate in gates.values():
    queue.put((gate.priority, gate))

students_left = are_students_left(gates.values())

while students_left:
    _, gate = queue.get()

    # If a gate has a guard, I want to let a student in
    if gate.has_guard():
        gate.let_student_in()

    else:
        # If a gate has no guards I need to add a free guard
        if guards:
            guard = guards.pop(0)
            gate.set_guard(guard)
            gate.let_student_in()

        # If there are no free guards at this stage, we don't have enough guards and exit the loop
        else:
            break

    # If no more students are waiting, put the guard back into the pool
    if gate.can_be_closed():
        guards.append(gate.remove_guard())
    
    # Add the gate back into the priority queue with its updated priority
    queue.put((gate.priority, gate))
    
    students_left = are_students_left(gates.values())

if guards:
    print("YES")
else:
    print("NO")

YES


## Space Complexity
Since we have $G$ guards and $M$ students that we store, we have a space complexity of $O(G + M)$.

## Time Complexity

We have a priority queue which we have to build once and then deque $N$ times, one for each gate.

* build once $O(n log n)$
* deque $n$ items $O(n log n)$. Dequeing one time takes $O(log n)$.

